In [47]:
import numpy as np
import datetime
from tqdm import tqdm
import pandas as pd
import pytz
import random
from concurrent.futures import ProcessPoolExecutor
import seaborn as sns
import sys
from skimage.morphology import rectangle, closing
from bb_behavior import db as bbdb;
import bb_behavior
import bb_utils
sys.path.append('/home/mi/rrszynka/mnt/janek/Beesbook-janek/Python-modules/') #For bee_helpers, file_helpers and cache
from bee_cache import Cache, CacheType, CacheFormat; c = Cache()

%matplotlib inline

In [48]:
detection_confidence_requirement = 0.2 # 0.2 // 0.99
offset_from_middle_seconds = datetime.timedelta(seconds=60)
meta = bb_utils.meta.BeeMetaInfo()

In [49]:
presences = c.load_multiple_day_caches(detection_confidence_requirement=detection_confidence_requirement) # 0.99 / 0.2


  0%|          | 0/62 [00:00<?, ?it/s]

loading PRESENCE-counts-2016-07-20_00_num_hours_24_int_size_120_conf_02_cams_0123.csv



  2%|▏         | 1/62 [00:01<01:07,  1.10s/it]

loading PRESENCE-counts-2016-07-21_00_num_hours_24_int_size_120_conf_02_cams_0123.csv



  3%|▎         | 2/62 [00:02<01:07,  1.13s/it]

...

loading PRESENCE-counts-2016-09-13_00_num_hours_24_int_size_120_conf_02_cams_0123.csv
loading PRESENCE-counts-2016-09-14_00_num_hours_24_int_size_120_conf_02_cams_0123.csv



 94%|█████████▎| 58/62 [00:49<00:00,  6.47it/s]

loading PRESENCE-counts-2016-09-15_00_num_hours_24_int_size_120_conf_02_cams_0123.csv
loading PRESENCE-counts-2016-09-16_00_num_hours_24_int_size_120_conf_02_cams_0123.csv
loading PRESENCE-counts-2016-09-17_00_num_hours_24_int_size_120_conf_02_cams_0123.csv



100%|██████████| 62/62 [00:49<00:00,  9.10it/s]

loading PRESENCE-counts-2016-09-18_00_num_hours_24_int_size_120_conf_02_cams_0123.csv
loading PRESENCE-counts-2016-09-19_00_num_hours_24_int_size_120_conf_02_cams_0123.csv
Collected 60/62 per-day caches.


In [ ]:
gaps_list = []
for (date, presence) in tqdm(presences):
    npdate = np.datetime64(date)
    gaps = make_gaps_for_presence(presence, date)
    gaps_list.append((date, gaps))


  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

 50%|█████     | 1/2 [00:08<00:08,  8.76s/it]

100%|██████████| 2/2 [00:14<00:00,  7.72s/it]

  0%|          | 0/2 [00:00<?, ?it/s]

  2%|▏         | 1/60 [00:26<25:46, 26.22s/it]

  0%|          | 0/2 [00:00<?, ?it/s]

 50%|█████     | 1/2 [00:09<00:09,  9.47s/it]

100%|██████████| 2/2 [00:15<00:00,  8.35s/it]

  0%|          | 0/2 [00:00<?, ?it/s]

  3%|▎         | 2/60 [00:54<26:00, 26.90s/it]

  0%|          | 0/2 [00:00<?, ?it/s]

 50%|█████     | 1/2 [00:09<00:09,  9.75s/it]

100%|██████████| 2/2 [00:15<00:00,  8.66s/it]

  0%|          | 0/2 [00:00<?, ?it/s]

  5%|▌         | 3/60 [01:25<26:37, 28.03s/it]

  0%|          | 0/2 [00:00<?, ?it/s]

 50%|█████     | 1/2 [00:47<00:47, 47.20s/it]

100%|██████████| 2/2 [01:06<00:00, 38.68s/it]

  0%|          | 0/2 [00:00<?, ?it/s]

  7%|▋         | 4/60 [02:48<41:42, 44.68s/it]

  0%|          | 0/2 [00:00<?, ?it/s]

 50%|█████     | 1/2 [00:47<00:47, 47.49

In [ ]:
len(gaps_list)

In [44]:
conf_string = str(detection_confidence_requirement).replace('.','')
for date, gaps in gaps_list:
    print(date)
    c.save(gaps, 'GAPS-'+str(date)+'_conf_'+conf_string, type=CacheType.gaps)

2016-07-20
2016-07-21


In [45]:
conf_string = str(detection_confidence_requirement).replace('.','')
aa = c.load('TESTGAPS-'+str(date)+'_conf_'+conf_string, type=CacheType.gaps)

In [46]:
aa.iloc[0]

bee_id                                                              9
age                                                                23
age_group                                                           4
exit                                              2016-07-21 01:37:00
entry                                             2016-07-21 01:59:30
gap_duration                                          0 days 00:22:30
exit_x                                                        214.994
exit_y                                                        89.4874
exit_frame                                       12042659669240907892
exit_closest_detection_timestamp     2016-07-21 01:37:00.079138+00:00
cam_id_exit                                                         3
hive_side_exit                                                      1
entry_x                                                       221.637
entry_y                                                       100.934
entry_frame         

In [29]:
gapsees=[]
for d, p in presences:
    gapsees.append(c.load('GAPS-'+str(date)+'_conf_'+conf_string, type=CacheType.gaps))

In [ ]:
gapsees[50]

In [ ]:
for presence in presences:
    c.load('GAPS-'+str(date)+'_conf_'+conf_string, type=CacheType.gaps)

In [13]:
def make_gaps_for_presence(presence, date):
    # 1. Binarize presence
    presence_bin = binarize_presence(presence)
    
    # 2. Detect gaps in presence (binarized)
    gaps = []
    for bee, bin_row in presence_bin.iterrows():
        row = presence.loc[bee]
        diff = np.diff(bin_row)
        exits = np.where(diff == -1)[0]
        entries = np.where(diff == 1)[0]

        if len(exits) == 0 or len(entries) == 0:
            continue

        # if first entry is before firs exit, start with second entry
        if entries[0] < exits[0]:
            entries = entries[1:]
            if len(entries) == 0:
                continue

        for exit, entry in zip(exits, entries):
            duration = (entry - exit)
            gaps.append((bee, exit, entry, duration))
            
    npdate = np.datetime64(date)
    gaps = pd.DataFrame(gaps)
    gaps.columns = ['bee_id', 'exit', 'entry', 'gap_duration']
    gaps.exit = gaps.exit.apply(interval_to_timepoint_for_current_date)
    gaps.entry = gaps.entry.apply(interval_to_timepoint_for_current_date)
    gaps.gap_duration = gaps.gap_duration.apply(lambda intervals: pd.Timedelta(minutes=intervals / 2))
    gaps.exit = gaps.exit.apply(to_datetime)
    gaps.entry = gaps.entry.apply(to_datetime)
    gaps['age'] = gaps.apply(row_to_age, axis=1)
    gaps = gaps.dropna() #TODO: some ages are nan, which and why? (or just discard)
    gaps['age_group'] = gaps.age.apply(lambda age: int(age) // 5) # Why does this throw warning?
    gaps = gaps[['bee_id', 'age', 'age_group', 'exit', 'entry', 'gap_duration']]

    # Add location, frame, cam and hive side information
    for mode in tqdm(['exit', 'entry']):
        new_info = make_dataframe_for_exits_entries(gaps, mode)
        gaps = pd.merge(gaps, new_info, on=['bee_id', mode])

    gaps['origin_for_exit'] = gaps.apply(origin_for_exit, axis=1)
    gaps['origin_for_entry'] = gaps.apply(origin_for_entry, axis=1)
    
    # Calculate breakpoint distances
    for mode in tqdm(['exit', 'entry']):
        origin_x = [x for (x,y) in gaps['origin_for_'+mode]]
        origin_y = [y for (x,y) in gaps['origin_for_'+mode]]

        diff_x = gaps[mode+'_x'] - origin_x
        diff_y = gaps[mode+'_y'] - origin_y

        d = pd.Series(np.linalg.norm([diff_x, diff_y], axis=0))

        gaps['hiveexit_distance_for_' + mode] = d
    
    return gaps   

In [ ]:
#TODO: save (hdf/pkl/csv?)

### Functions

In [1]:
def make_dataframe_for_exits_entries(gaps, mode):
    # Transform into a dict 
    gaps_dict = {}
    for bee in gaps.bee_id.unique():
        tsx = gaps[gaps.bee_id==bee][mode]
        gaps_dict[int(bee)] = tsx

    
    subset = gaps_dict #gaps_head_dict // gaps_dict // bee_exits_dict
    all_data = []

    with ProcessPoolExecutor(max_workers=16) as pool:
        items = list(subset.items()) # list of tuples 
        items = [(bee_id, timestamp, mode) for (bee_id, timestamp) in items]
        for sub_df in pool.map(get_data_for_bee_id_timestamps, items):
            all_data += sub_df
            
    all_data = [row for row in all_data if row != None]
    all_data_df = pd.DataFrame(all_data, columns=["bee_id", mode, mode+"_x", mode+"_y", mode+'_frame', mode+'_closest_detection_timestamp', "cam_id_"+mode])

    def to_hive_side(cam_id):
        if cam_id is None or pd.isnull(cam_id):
            return 2
        return cam_id // 2
    
    all_data_df['hive_side_'+mode] = all_data_df['cam_id_'+mode].apply(to_hive_side).astype(np.int)

    
    return all_data_df

In [2]:
def get_data_for_bee_id_timestamps(args):
    bee_id, timestamps, mode = args
    offset = offset_from_middle_seconds
    
    with bb_behavior.db.DatabaseCursorContext() as cursor:
        cursor.execute("PREPARE fetch_cam_id AS "
                       "SELECT cam_id, x_pos_hive, y_pos_hive, frame_id, timestamp, ABS(EXTRACT(epoch FROM (timestamp - $4))) as diff FROM bb_detections_2016_stitched "
                       "WHERE bee_id = $1 "
                       "AND timestamp >= $2 and timestamp <= $3 "
                       "ORDER BY diff ASC "
                       "LIMIT 10 "
                      )
        
        def fetch_cam_id_and_location(bee_id, ts, cursor):
            cursor.execute("EXECUTE fetch_cam_id(%s, %s, %s, %s)", (bee_id, ts - offset, ts + offset, ts))
            data = cursor.fetchall()
            if data is None or len(data) == 0:
                return None
            
            any_cam_id = data[0][0]
            locations = np.array([d[1:3] for d in data if d[0] == any_cam_id])
            
            frame_id = data[0][3]
            timestamp = data[0][4]
            
            
            return bee_id, ts, np.mean(locations, axis=0)[0], np.mean(locations, axis=0)[1], frame_id, timestamp, any_cam_id
        

        results = []
        for ts in timestamps:
            results.append(fetch_cam_id_and_location(bee_id, ts, cursor))
        return results

In [3]:
#From pres_to_gaps
def binarize_presence(presence_for_day_df):
    # TODO: running this on an already-binarized series will return all zeros,
    # (and that's not what we want) - changing the threshold solves it, but try sth else

    ys = presence_for_day_df.copy()
    index = ys.index
    if ys[ys>1].sum().sum() == 0:
        #Consider this already binarized, make no changes
        return ys

    ys[ys>90] = 90
    ys[ys>45] = 90
    ys[ys<=45] = 0 #TODO: consult: what should the threshold be
    ys[ys==90] = 1
    ys = closing(ys, rectangle(1,15))
    ys = pd.DataFrame(ys, index=index)
    ys.index.name = 'bee_id'
    return ys

#%%
def plot_presence(a):
    plt.figure(figsize=(26,7))
    plt.title("Presence")
    axes = plt.gca()
    plt.scatter(np.arange(0,len(a)), a, s=3)

#%%
def get_frame_id_for_timestamp_for_current_bee(dt):
    bee_id = int(bee)
    get_frame_id_for_bee_id_and_timestamp(bee_id, dt)

#%%
def get_frame_id_from_row(row):
    bee_id = row.bee
    dt = row.exit
    return get_frame_id_for_bee_id_and_timestamp(bee_id, dt)

#%%
def interval_to_timepoint_for_current_date(interval_number):
    # Date must already be np.datetime64, so as not to convert here (executed for every row)
    return npdate + np.timedelta64(int(interval_number)*30, 's')

#%%
def row_to_age(row):
    bb_id = bb_utils.ids.BeesbookID.from_dec_9(row.bee_id)
    age = meta.get_age(bb_id, row.exit).days
    return age

#%%
def origin_for_exit(row):
    origin = np.array([0, 250]) if row['hive_side_exit'] == 1 else np.array([350, 250])
    return origin

#%%
def origin_for_entry(row):
    origin = np.array([0, 250]) if row['hive_side_entry'] == 1 else np.array([350, 250])
    return origin


In [4]:
#Unused here, potentially useful elsewhere 
def to_datetime(ts):
    if ts.tzinfo is None or ts.tzinfo.utcoffset(ts) is None:
        return ts
    else:
        return pytz.UTC.localize(pd.Timestamp(ts).to_pydatetime(), pytz.UTC)